### 📂 1. Carregando os dados

In [4]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
# Carregar o dataset
df = pd.read_csv("data/noticias_sinteticas.csv")

# Visualizar as primeiras linhas
df.head()

,texto,category
0,O ministro anunciou cortes no orçamento federal.,política
1,A equipe conquistou o campeonato estadual.,esporte
2,O técnico anunciou a escalação para o próximo ...,esporte
3,Houve protestos contra a nova legislação aprov...,política
4,Houve protestos contra a nova legislação aprov...,política


### 🧹 2. Pré-processamento de texto

In [6]:
nltk.download('stopwords')
stopwords_pt = set(stopwords.words('portuguese'))

def preprocessar_texto(texto):
    texto = texto.lower()
    texto = re.sub(r'[^\w\s]', '', texto)  # Remove pontuação
    palavras = texto.split()
    palavras = [p for p in palavras if p not in stopwords_pt]
    return ' '.join(palavras)

# Aplicar o pré-processamento
df['texto_limpo'] = df['texto'].apply(preprocessar_texto)

# Visualizar resultado
df[['texto', 'texto_limpo']].head()

[nltk_data] Downloading package stopwords to C:\Users\Jorge
[nltk_data]     Maques/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,texto,texto_limpo
0,O ministro anunciou cortes no orçamento federal.,ministro anunciou cortes orçamento federal
1,A equipe conquistou o campeonato estadual.,equipe conquistou campeonato estadual
2,O técnico anunciou a escalação para o próximo ...,técnico anunciou escalação próximo jogo
3,Houve protestos contra a nova legislação aprov...,protestos contra nova legislação aprovada
4,Houve protestos contra a nova legislação aprov...,protestos contra nova legislação aprovada


### 🧮 3. Vetorização com Bag of Words (BoW)

In [7]:
# Criar o vetor de contagem (BoW)
vectorizer_bow = CountVectorizer()
X_bow = vectorizer_bow.fit_transform(df['texto_limpo'])

print(f"Dimensão da matriz BoW: {X_bow.shape}")

# Exemplo: mostrar as 10 primeiras palavras do vocabulário
print("Primeiras 10 palavras do vocabulário BoW:")
print(list(vectorizer_bow.get_feature_names_out())[:10])

Dimensão da matriz BoW: (500, 85)
Primeiras 10 palavras do vocabulário BoW:
['acréscimos', 'adiada', 'alta', 'anunciaram', 'anunciou', 'aplicativo', 'apresentou', 'aprovada', 'aprovou', 'ator']


### 📊 4. Vetorização com TF-IDF

In [8]:
# Criar o vetor TF-IDF
vectorizer_tfidf = TfidfVectorizer()
X_tfidf = vectorizer_tfidf.fit_transform(df['texto_limpo'])

print(f"Dimensão da matriz TF-IDF: {X_tfidf.shape}")

# Exemplo: mostrar os scores TF-IDF das 10 primeiras palavras do vocabulário
print("Primeiras 10 palavras do vocabulário TF-IDF:")
print(list(vectorizer_tfidf.get_feature_names_out())[:10])

Dimensão da matriz TF-IDF: (500, 85)
Primeiras 10 palavras do vocabulário TF-IDF:
['acréscimos', 'adiada', 'alta', 'anunciaram', 'anunciou', 'aplicativo', 'apresentou', 'aprovada', 'aprovou', 'ator']


### 🔍 5. Comparação entre Bag of Words e TF-IDF

In [9]:
# Escolher um índice para exemplo
idx = 0

print("Texto original:")
print(df.loc[idx, 'texto'])

print("\nTexto pré-processado:")
print(df.loc[idx, 'texto_limpo'])

# Obter vocabulário das palavras para BoW e TF-IDF
voc_bow = vectorizer_bow.get_feature_names_out()
voc_tfidf = vectorizer_tfidf.get_feature_names_out()

# Vetor BoW do documento escolhido (esparso para denso)
vec_bow = X_bow[idx].toarray().flatten()

# Vetor TF-IDF do documento escolhido
vec_tfidf = X_tfidf[idx].toarray().flatten()

# Mostrar palavras com valor > 0 e seus respectivos scores BoW e TF-IDF
print("\nPalavras e valores BoW:")
for palavra, valor in zip(voc_bow, vec_bow):
    if valor > 0:
        print(f"{palavra}: {valor}")

print("\nPalavras e valores TF-IDF:")
for palavra, valor in zip(voc_tfidf, vec_tfidf):
    if valor > 0:
        print(f"{palavra}: {valor:.4f}")

Texto original:
O ministro anunciou cortes no orçamento federal.

Texto pré-processado:
ministro anunciou cortes orçamento federal

Palavras e valores BoW:
anunciou: 1
cortes: 1
federal: 1
ministro: 1
orçamento: 1

Palavras e valores TF-IDF:
anunciou: 0.3496
cortes: 0.4684
federal: 0.4684
ministro: 0.4684
orçamento: 0.4684


### ✅ Conclusão: Bag of Words vs. TF-IDF

Neste estudo, comparamos duas das principais técnicas de vetorização de texto:

- **Bag of Words (BoW)** representa cada documento por meio da contagem de palavras, sem considerar o contexto ou a relevância dos termos no corpus.
- **TF-IDF** (Term Frequency–Inverse Document Frequency) vai além, atribuindo pesos que refletem a importância das palavras, penalizando termos comuns e valorizando termos raros.

### Exemplo prático

- No texto: *"O ministro anunciou cortes no orçamento federal"*, todas as palavras receberam valor 1 no BoW, pois cada uma aparece apenas uma vez.
- Já no TF-IDF, a palavra "anunciou" teve um peso menor, indicando que ela aparece com mais frequência no conjunto de documentos. Por outro lado, termos como "orçamento" e "federal" tiveram pesos maiores, indicando que são mais informativos para esse texto.

### Conclusão

- **BoW** é simples, rápido e pode ser suficiente para modelos mais básicos.
- **TF-IDF** geralmente oferece melhores resultados em tarefas como classificação e busca textual, por capturar a relevância dos termos.
